# Importation du fichier csv :

In [1]:
data = read.csv("personnes.csv", 
                stringsAsFactors = FALSE, # pour éviter de travailler avec des Factors et faciliter les traitements
                fileEncoding = "UTF-8", # spécifier l'encodage du fichier
                na.strings=c("","NA")) # afin qu'il considère bien les "cases vides" comme des NA
head(data, 10)

prenom,email,date_naissance,pays,taille
Leila,leila@example.com,23/01/1990,France,1.49m
Samuel,samuel_329@example.com,20/09/2001,NA,1.67m
Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m
samuël,samuel_329@example.com,NA,Bénin,1.45m


# 1. Détecter les erreurs

Détecter les valeurs manquantes :

In [2]:
sapply(data, function(y) sum(is.na(y)))

prenom          email date_naissance           pays         taille 
             0              0              1              1              0

Détecter les doublons :

In [3]:
duplicated(data$email)

[1] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE

In [4]:
data[duplicated(data$email) | duplicated(data$email, fromLast=TRUE),]

,prenom,email,date_naissance,pays,taille
2,Samuel,samuel_329@example.com,20/09/2001,NA,1.67m
7,samuël,samuel_329@example.com,NA,Bénin,1.45m


# 2. Traiter les erreurs

## Traiter la variable Pays

Pour les pays nous avions un problème majeur qui était la valeur 24 , à la 5ème ligne. La première approche pourrait être de considérer que l'ensemble de mes pays doivent être des chaines de caractères et donc toutes les valeurs numériques doivent être remplacées par des valeurs manquantes. Mais pour faire la chose encore plus "proprement", on pourrait définir une liste de pays valides, et remplacer toutes les valeurs qui ne concordent pas avec la liste par des valeurs manquantes :

In [5]:
VALID_COUNTRIES = c("France", "Côte d'ivoire", "Madagascar", "Bénin", "Allemagne", "USA")
mask = !data$pays %in% VALID_COUNTRIES
data[mask, 'pays'] = NA

head(data, 10)

prenom,email,date_naissance,pays,taille
Leila,leila@example.com,23/01/1990,France,1.49m
Samuel,samuel_329@example.com,20/09/2001,NA,1.67m
Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
Hanna,hanna2019@supermail.eu,01/01/1970,NA,3.45m
samuël,samuel_329@example.com,NA,Bénin,1.45m


## Traiter les emails

Au tour des emails maintenant ! Le problème avec cette colonne, c'est qu'il y a parfois 2 adresses email par ligne. Nous ne souhaitons prendre que la première. Nous pouvons là aussi utiliser les outils fournies par R pour faire cela :

In [6]:
data$email = sapply(strsplit(data$email,","), getElement, 1)

head(data, 10)

prenom,email,date_naissance,pays,taille
Leila,leila@example.com,23/01/1990,France,1.49m
Samuel,samuel_329@example.com,20/09/2001,NA,1.67m
Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
Marc,marco23@example.com,10/02/1978,France,1.65m
Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
Hanna,hanna2019@supermail.eu,01/01/1970,NA,3.45m
samuël,samuel_329@example.com,NA,Bénin,1.45m


## Traiter les tailles

Nous aurons besoin d'effectuer deux opérations :

1. convertir les tailles en nombre décimal. Nous en profiterons pour remplacer l'ensemble des tailles non conformes par des valeurs manquantes
2. remplacer les valeurs manquantes par la moyenne de notre échantillon

Pour la première étape, comme l'ensemble de nos valeurs sont censé avoir pour format X.XXm (avec X correspondant à un chiffre), on peut par exemple sélectionner uniquement les 4 premiers caractères et convertir l'ensemble de la colonne en numérique. Ce faisant, les valeurs non valides seront automatiquement remplacées par des valeurs manquantes lors de l'utilisation de la fonction as.numeric :

In [7]:
data$taille = substr(data$taille, 1, 4)
data$taille = as.numeric(data$taille)

Warning message in eval(expr, envir, enclos):
"NAs introduits lors de la conversion automatique"

Ensuite, il faut à présent remplacer les valeurs manquantes par la moyenne de la variable. Cette simple ligne permettra de faire cela :

In [8]:
data$taille[is.na(data$taille)] <- mean(data$taille, na.rm=TRUE)

head(data, 10)

prenom,email,date_naissance,pays,taille
Leila,leila@example.com,23/01/1990,France,1.490000
Samuel,samuel_329@example.com,20/09/2001,NA,1.670000
Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,1.841667
Marc,marco23@example.com,10/02/1978,France,1.650000
Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.340000
Hanna,hanna2019@supermail.eu,01/01/1970,NA,3.450000
samuël,samuel_329@example.com,NA,Bénin,1.450000


## Traiter les dates

Dernier sujet à traiter et pas des moindres : les dates ! Ici, nous allons simplement spécifier un certain format de date et transformer notre variable dans un type adéquat :

In [9]:
data$date_naissance = as.Date(data$date_naissance , "%d/%m/%Y")

head(data, 10)

prenom,email,date_naissance,pays,taille
Leila,leila@example.com,1990-01-23,France,1.490000
Samuel,samuel_329@example.com,2001-09-20,NA,1.670000
Radia,choupipoune@supermail.eu,NA,Côte d'ivoire,1.841667
Marc,marco23@example.com,1978-02-10,France,1.650000
Heri,helloworld@supermail.eu,2008-03-05,Madagascar,1.340000
Hanna,hanna2019@supermail.eu,1970-01-01,NA,3.450000
samuël,samuel_329@example.com,NA,Bénin,1.450000
